In [64]:
import pandas as pd 
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyOAuth 
import pickle
scope1 = "user-library-read"
from spotipy.oauth2 import SpotifyClientCredentials
sp =spotipy.Spotify(auth_manager= SpotifyOAuth(client_id = "15fd015d4da742d0bf7da9c6ed3120c7", client_secret = "349c5181636e4e3cbb974b051db425fd", redirect_uri = "http://localhost", scope= scope1))

In [65]:
filename = 'model.pkl'
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

In [66]:
liked_songs = []
results = sp.current_user_saved_tracks()
while results:
    for item in results['items']:
        track = item['track']
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        album_name = track['album']['name']
        song_id = track['id']
        row = {"track_name" :track_name, "artist_name" : artist_name, "album_name" : album_name, "song_id": song_id}
        liked_songs.append(row)
    results = sp.next(results)
output = pd.DataFrame(liked_songs)

In [67]:
#gather data using soptify API
song_ana = pd.DataFrame()
for i in range (len(output)) :
    analysis = sp.audio_features(output.at[i, "song_id"])
    analy = pd.DataFrame(analysis)
    song_ana = pd.concat([song_ana, analy])
song_ana = song_ana.drop(["type", "duration_ms", "time_signature", "mode"], axis= 1).reset_index().drop(["index"], axis= 1)

In [68]:

# Normalize the DataFrame to be between 0 and 1
min_value = song_ana[["tempo"]].min()
max_value = song_ana[["tempo"]].max()
song_ana["tempo"] = (song_ana[["tempo"]] - min_value) / (max_value - min_value)

min_value = song_ana[["loudness"]].min()
max_value = song_ana[["loudness"]].max()
song_ana["loudness"] = (song_ana[["loudness"]] - min_value) / (max_value - min_value)

min_value = song_ana[["key"]].min()
max_value = song_ana[["key"]].max()
song_ana["key"] = (song_ana[["key"]] - min_value) / (max_value - min_value)


*starting to classifed*

In [69]:
#target cloumn
column_select = song_ana[["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]

In [77]:
song_ana["prediciton"] = loaded_model.predict(column_select)

In [78]:
song_ana

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,track_href,analysis_url,prediciton
0,0.718,0.475,0.636364,0.686736,0.0611,0.845000,0.801000,0.1050,0.678,0.640867,4xk70Qur2QeRmWIzWPC63V,spotify:track:4xk70Qur2QeRmWIzWPC63V,https://api.spotify.com/v1/tracks/4xk70Qur2QeR...,https://api.spotify.com/v1/audio-analysis/4xk7...,3
1,0.854,0.682,0.727273,0.731665,0.0333,0.298000,0.000024,0.1160,0.842,0.296070,6xAtGKQApLCPft6DYR6fFK,spotify:track:6xAtGKQApLCPft6DYR6fFK,https://api.spotify.com/v1/tracks/6xAtGKQApLCP...,https://api.spotify.com/v1/audio-analysis/6xAt...,2
2,0.463,0.566,0.727273,0.846303,0.1450,0.196000,0.000000,0.2840,0.219,0.245445,6z1kLsntE7FuzKZHZWrXYN,spotify:track:6z1kLsntE7FuzKZHZWrXYN,https://api.spotify.com/v1/tracks/6z1kLsntE7Fu...,https://api.spotify.com/v1/audio-analysis/6z1k...,3
3,0.707,0.681,0.000000,0.902880,0.0668,0.063200,0.000005,0.0322,0.646,0.396306,4DHcnVTT87F0zZhRPYmZ3B,spotify:track:4DHcnVTT87F0zZhRPYmZ3B,https://api.spotify.com/v1/tracks/4DHcnVTT87F0...,https://api.spotify.com/v1/audio-analysis/4DHc...,1
4,0.681,0.580,0.000000,0.841069,0.0686,0.650000,0.000003,0.1590,0.486,0.266152,7GLn0324YKGYxbYN7GH8Ip,spotify:track:7GLn0324YKGYxbYN7GH8Ip,https://api.spotify.com/v1/tracks/7GLn0324YKGY...,https://api.spotify.com/v1/audio-analysis/7GLn...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,0.600,0.663,0.272727,0.858193,0.0477,0.225000,0.000000,0.1190,0.286,0.415547,4nVBt6MZDDP6tRVdQTgxJg,spotify:track:4nVBt6MZDDP6tRVdQTgxJg,https://api.spotify.com/v1/tracks/4nVBt6MZDDP6...,https://api.spotify.com/v1/audio-analysis/4nVB...,3
1671,0.653,0.822,0.000000,0.886784,0.0330,0.000210,0.004700,0.0838,0.762,0.395993,6BnONjR7itGMEqwxKTIlRM,spotify:track:6BnONjR7itGMEqwxKTIlRM,https://api.spotify.com/v1/tracks/6BnONjR7itGM...,https://api.spotify.com/v1/audio-analysis/6BnO...,1
1672,0.730,0.770,0.727273,0.879009,0.0298,0.138000,0.000000,0.0473,0.507,0.314535,1eq1wUnLVLg4pdEfx9kajC,spotify:track:1eq1wUnLVLg4pdEfx9kajC,https://api.spotify.com/v1/tracks/1eq1wUnLVLg4...,https://api.spotify.com/v1/audio-analysis/1eq1...,2
1673,0.554,0.321,0.818182,0.784098,0.0280,0.893000,0.000000,0.0996,0.288,0.503120,6QPKYGnAW9QozVz2dSWqRg,spotify:track:6QPKYGnAW9QozVz2dSWqRg,https://api.spotify.com/v1/tracks/6QPKYGnAW9Qo...,https://api.spotify.com/v1/audio-analysis/6QPK...,3


In [79]:
calm = song_ana[song_ana['prediciton'] == 0 ]
happy = song_ana[song_ana['prediciton'] == 1 ]
energy = song_ana[song_ana['prediciton'] == 2 ]
sad = song_ana[song_ana['prediciton'] == 3 ]

In [80]:
len(calm)

41

In [81]:
len(happy)

141

In [82]:
len(energy)

507

In [76]:
len(sad)

986